# Исследование влияния преобразования изображений документов на качество детекции и распознавания штрих-кодов
- Автор: Кирилл Киселев
- Дата начала: 05.04.2022
- Описание исследования: в наборе данных имеется 205 сканов документов ТОРГ-12 разного качества. На каждом скане имеется штрих-код, который необходимо детектировать и распознать. Тестирование прототипа производилось спрева на "сырых" данных - изображения не подвергались преобразовниям кроме обрезания области детекции. При ттестирования прототипа штрих-код не был обнаружен на 11 документов. Задача исследования подобрать типы преобразований необходимые для распознваваний штрих-кодов на 100% изображений.

## 1. Проблемные изображения

## 2. Загрузка библиотек

In [2]:
import cv2
import numpy as np
import matplotlib as plt
import os
import math
from PIL import Image
from pyzbar import pyzbar

## 3. Глобальные переменные

In [3]:
!pwd

/home/kirillk/PycharmProjects/useful_notebooks


In [4]:
FILE_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases_2/'

## 4. Необходимые функции

In [5]:
def draw_barcode(decoded, image):
    image = cv2.rectangle(image, (decoded.rect.left, decoded.rect.top),
                          (decoded.rect.left + decoded.rect.width, decoded.rect.top + decoded.rect.height),
                          color=(0, 255, 0),
                          thickness=5)
    return image

In [6]:
def decode(image):
    # decodes all barcodes from an image
    decoded_objects = pyzbar.decode(image)
    
    for obj in decoded_objects:
        # draw the barcode
        print(f"Обнаружен штрих-код:\n{obj}")
        image = draw_barcode(obj, image)
        # print barcode type & data
        # print("Тип:", obj.type)
        print("Данные:", obj.data)
        print()

    return len(decoded_objects), image

## 5. Распознавание штрих-кода

In [7]:
jpg_list = []

In [8]:
for file in os.listdir(FILE_PATH):
    if file.endswith('.jpg'):
        jpg_list.append(file)

In [9]:
jpg_list = sorted(jpg_list)

In [10]:
bad_barcodes_list = []

In [13]:
for file in jpg_list:
    print(f'----------------{file}----------------')
    image = cv2.imread(FILE_PATH + file)
    image = image[0:image.shape[0], 0:int(image.shape[1]*0.25)]
#     image = cv2.GaussianBlur(image, (7, 7), 0)    
    
    bar_code_qty, img = decode(image)
    if bar_code_qty == 0:
        bad_barcodes_list.append(file)
        
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

----------------001.jpg----------------
----------------002.jpg----------------
----------------003.jpg----------------
----------------004.jpg----------------
----------------005.jpg----------------
----------------006.jpg----------------
----------------007.jpg----------------
----------------008.jpg----------------
----------------009.jpg----------------
----------------010.jpg----------------
----------------011.jpg----------------
----------------012.jpg----------------
Обнаружен штрих-код:
Decoded(data=b'2810142821818', type='EAN13', rect=Rect(left=76, top=70, width=157, height=16), polygon=[Point(x=76, y=77), Point(x=76, y=81), Point(x=233, y=86), Point(x=233, y=70), Point(x=77, y=71)], quality=15, orientation='UP')
Данные: b'2810142821818'

----------------013.jpg----------------
Обнаружен штрих-код:
Decoded(data=b'2810142821825', type='EAN13', rect=Rect(left=88, top=48, width=157, height=12), polygon=[Point(x=88, y=49), Point(x=88, y=55), Point(x=165, y=59), Point(x=245, y=60)

In [39]:
sorted(bad_barcodes_list)

['001.jpg',
 '002.jpg',
 '003.jpg',
 '004.jpg',
 '005.jpg',
 '006.jpg',
 '007.jpg',
 '008.jpg',
 '009.jpg',
 '010.jpg',
 '011.jpg']

## 5. Применение Гауссовского блюра на нераспознанные документы

In [6]:
# исходный список нараспознанных документов
bad_barcodes_list = ['038.jpg', '039.jpg', '040.jpg',
                     '041.jpg', '042.jpg', '043.jpg',
                     '044.jpg', '095.jpg', '146.jpg',
                     '157.jpg', '159.jpg']

In [7]:
# список документов с нераспознанным штрих-кодом
bad_barcodes_list_1 = []

In [18]:
for file in bad_barcodes_list:
    print(f'----------------{file}----------------')
    image = cv2.imread(FILE_PATH + file)
    image = cv2.GaussianBlur(image, (7, 7), 0)
    
    bar_code_qty, img = decode(image)
    if bar_code_qty == 0:
        bad_barcodes_list_1.append(file)
    

----------------010.jpg----------------
----------------003.jpg----------------
----------------004.jpg----------------
----------------007.jpg----------------
----------------011.jpg----------------
----------------009.jpg----------------
----------------008.jpg----------------
----------------005.jpg----------------
----------------002.jpg----------------
----------------001.jpg----------------
----------------006.jpg----------------


In [9]:
bad_barcodes_list_1

['095.jpg', '146.jpg', '157.jpg', '159.jpg']

### Вывод
<br>Гауссовский блюр позволил распознать 7 изображений из 11.

## 6. Не распознанные изображения после Гауссовского блюра

- 095.jpg - непонятно, почему не распознано
- 146.jpg - шум
- 157.jpg - блюр
- 159.jpg - блюр

## 7. Случай с "непонятным" изображением 095.jpg

In [10]:
image = cv2.imread(FILE_PATH + bad_barcodes_list_1[0])
image = image[0:image.shape[0], 0:int(image.shape[1]*0.25)]
image = cv2.blur(image,(7, 7), 0)
barcode_list, img = decode(image)

Обнаружен штрих-код:
Decoded(data=b'2860591580422', type='EAN13', rect=Rect(left=413, top=348, width=662, height=66), polygon=[Point(x=413, y=377), Point(x=728, y=413), Point(x=1075, y=414), Point(x=1075, y=370), Point(x=1074, y=348), Point(x=749, y=348), Point(x=728, y=349)], quality=69, orientation='UP')
Данные: b'2860591580422'



### Вывод
<br>Замена Гауссовского блюра на простой блюр позволило распознать изображение 095.jpg.

## 8. Случай с зашумленным изображением 146.jpg

In [11]:
image = cv2.imread(FILE_PATH + bad_barcodes_list_1[1])
image = image[0:image.shape[0], 0:int(image.shape[1]*0.25)]
image = cv2.GaussianBlur(image, (7, 7), 0)
# image = cv2.blur(image, (5, 5), 0)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image = cv2.threshold(image, 130, 255, cv2.THRESH_BINARY)

_, img = decode(image)

# cv2.imshow("Image", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Обнаружен штрих-код:
Decoded(data=b'2860601473409', type='EAN13', rect=Rect(left=307, top=285, width=675, height=56), polygon=[Point(x=307, y=287), Point(x=307, y=291), Point(x=309, y=341), Point(x=634, y=341), Point(x=980, y=340), Point(x=982, y=308), Point(x=982, y=306), Point(x=980, y=286), Point(x=634, y=285)], quality=64, orientation='UP')
Данные: b'2860601473409'



### Вывод
<br>Зашумленное изображение можно распознать применив либо `blur` либо `Gaussian` затем<br>
`image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image = cv2.threshold(image, 130, 255, cv2.THRESH_BINARY)`

## 9. Случай с размытым изображением 157.jpg

In [12]:
threshold_list_157 = []

In [13]:
%%time

for i in range (180, 200):
    image = cv2.imread(FILE_PATH + bad_barcodes_list_1[2])
    image = cv2.GaussianBlur(image, (1, 1), 0)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image = cv2.threshold(image, i, 255, cv2.THRESH_BINARY)
    
    barcode_qty, img = decode(image)
    if barcode_qty > 0:
        threshold_list_157.append(i)

Обнаружен штрих-код:
Decoded(data=b'2860610590685', type='EAN13', rect=Rect(left=252, top=177, width=676, height=20), polygon=[Point(x=252, y=181), Point(x=581, y=197), Point(x=928, y=196), Point(x=928, y=180), Point(x=583, y=177)], quality=14, orientation='UP')
Данные: b'2860610590685'

Обнаружен штрих-код:
Decoded(data=b'2860610590685', type='EAN13', rect=Rect(left=251, top=174, width=677, height=46), polygon=[Point(x=251, y=181), Point(x=928, y=220), Point(x=928, y=174), Point(x=252, y=175)], quality=21, orientation='UP')
Данные: b'2860610590685'

Обнаружен штрих-код:
Decoded(data=b'2860610590685', type='EAN13', rect=Rect(left=251, top=175, width=678, height=44), polygon=[Point(x=251, y=179), Point(x=251, y=181), Point(x=580, y=219), Point(x=928, y=198), Point(x=929, y=182), Point(x=928, y=178), Point(x=581, y=175), Point(x=252, y=175)], quality=26, orientation='UP')
Данные: b'2860610590685'

Обнаружен штрих-код:
Decoded(data=b'2860610590685', type='EAN13', rect=Rect(left=251, top=1

In [14]:
threshold_list_157

[189, 190, 191, 192, 193, 194, 195, 196, 197]

In [15]:
image = cv2.imread(FILE_PATH + bad_barcodes_list_1[2])
image = cv2.GaussianBlur(image, (1, 1), 0)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image = cv2.threshold(image, threshold_list_157[-1], 255, cv2.THRESH_BINARY)
    
_, img = decode(image)

# cv2.imshow("Image", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Обнаружен штрих-код:
Decoded(data=b'2860610590685', type='EAN13', rect=Rect(left=251, top=181, width=678, height=38), polygon=[Point(x=251, y=181), Point(x=579, y=219), Point(x=929, y=218), Point(x=929, y=182)], quality=10, orientation='UP')
Данные: b'2860610590685'



## 10. Случай с размытым изображением 159.jpg

In [16]:
threshold_list_159 = []

In [17]:
%%time

for i in range (180, 200):
    image = cv2.imread(FILE_PATH + bad_barcodes_list_1[3])
    image = cv2.GaussianBlur(image, (1, 1), 0)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image = cv2.threshold(image, i, 255, cv2.THRESH_BINARY)
    
    barcode_qty, img = decode(image)
    if barcode_qty > 0:
        threshold_list_159.append(i)

Обнаружен штрих-код:
Decoded(data=b'2860610411164', type='EAN13', rect=Rect(left=231, top=141, width=655, height=7), polygon=[Point(x=231, y=141), Point(x=231, y=145), Point(x=233, y=147), Point(x=886, y=148), Point(x=886, y=142)], quality=8, orientation='UP')
Данные: b'2860610411164'

Обнаружен штрих-код:
Decoded(data=b'2860610411164', type='EAN13', rect=Rect(left=231, top=146, width=655, height=4), polygon=[Point(x=231, y=147), Point(x=233, y=149), Point(x=569, y=150), Point(x=886, y=148), Point(x=886, y=146), Point(x=568, y=146)], quality=9, orientation='UP')
Данные: b'2860610411164'

Обнаружен штрих-код:
Decoded(data=b'2860610411164', type='EAN13', rect=Rect(left=230, top=143, width=656, height=7), polygon=[Point(x=230, y=143), Point(x=231, y=149), Point(x=569, y=150), Point(x=886, y=148)], quality=7, orientation='UP')
Данные: b'2860610411164'

Обнаружен штрих-код:
Decoded(data=b'2860610411164', type='EAN13', rect=Rect(left=230, top=142, width=656, height=8), polygon=[Point(x=230, 

In [18]:
threshold_list_159

[187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]

In [19]:
image = cv2.imread(FILE_PATH + bad_barcodes_list_1[3])
image = cv2.GaussianBlur(image, (1, 1), 0)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image = cv2.threshold(image, threshold_list_159[-3], 255, cv2.THRESH_BINARY)
    
_, img = decode(image)

# cv2.imshow("Image", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Обнаружен штрих-код:
Decoded(data=b'2860610411164', type='EAN13', rect=Rect(left=229, top=117, width=658, height=31), polygon=[Point(x=229, y=131), Point(x=229, y=137), Point(x=230, y=147), Point(x=570, y=148), Point(x=887, y=142), Point(x=887, y=118), Point(x=230, y=117)], quality=29, orientation='UP')
Данные: b'2860610411164'



In [20]:
# пересечени списка настроек порога
sorted(list(set(threshold_list_157) & set(threshold_list_159)))

[189, 190, 191, 192, 193, 194, 195, 196, 197]

### Вывод
<br>Сильно-размытое изображение можно распознать применив либо `Gaussian` с матрицей (1, 1)затем<br>
`image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image = cv2.threshold(image, n, 255, cv2.THRESH_BINARY)`<br>
где **n** лежит в диапазоне [189, 190, 191, 192, 193, 194, 195, 196, 197] мы применили n=197

## 11. Выводы

1. Для обработки изображений перед детекцией и распознаванием штрих-кода стоит применить трансформацию Гауссовского блюра с матрицей 7х7 `image = cv2.GaussianBlur(image, (7, 7), 0)` судя по тестовым данным такое преобразование примененное к каждому изображению повышает точность детекции штрих-кода до **98%**.
2. После применения Гауссовского блюра (п.1) прототип распознал 201 изображение из 205. В этом исследовании были подобраны преобразования для оставшихся 4 изображений - 095.jpg, 146.jpg, 157.jpg, 159.jpg.
3. Изображение 095.jpg - качественное изображение, странно, что тетектор штрих-кодов на нем не сработал. Для детекции было применено преобразование обыкновенного блюра с матрицей 7х7 `image = cv2.blur(image,(7, 7), 0)` после чего штрих-код был распознан.
4. Изображение 146.jpg имеет множество артефактов от печати, для него были применены следующие преобразования<br> 
`image = cv2.blur(image, (5, 5), 0)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image = cv2.threshold(image, 130, 255, cv2.THRESH_BINARY)`

5. Изображения 157.jpg, 159.jpg представляют из тебя размытые парактически нечитаемые сканы. Большая часть информации на них потеряна, человеку ее не прочитать. Однако удалось подобрать преобразвания, которые позволяют детектировать и распознать штрихкоды. Эти преобразования следующие<br>
`image = cv2.imread(FILE_PATH + bad_barcodes_list_1[3])
image = cv2.GaussianBlur(image, (1, 1), 0)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image = cv2.threshold(image, 197, 255, cv2.THRESH_BINARY)`

## 12. Примеры визуализации

### Применение Гауссовского блюра с ядром 7х7

In [23]:
image = cv2.imread(FILE_PATH + bad_barcodes_list[0])
image = image[0:image.shape[0], 0:int(image.shape[1]*0.25)]
image = cv2.GaussianBlur(image, (7, 7), 0)

_, img = decode(image)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Применение обычного блюра

In [25]:
image = cv2.imread(FILE_PATH + bad_barcodes_list_1[0])
image = image[0:image.shape[0], 0:int(image.shape[1]*0.25)]
# image = cv2.blur(image,(7, 7), 0)

barcode_list, img = decode(image)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Применение Гауссовского блюра, перевода картинки в черно-белый режим с добавлением бинаризации по порогу

In [27]:
image = cv2.imread(FILE_PATH + bad_barcodes_list_1[1])
image = image[0:image.shape[0], 0:int(image.shape[1]*0.25)]
image = cv2.GaussianBlur(image, (7, 7), 0)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image = cv2.threshold(image, 130, 255, cv2.THRESH_BINARY)

_, img = decode(image)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Обнаружен штрих-код:
Decoded(data=b'2860601473409', type='EAN13', rect=Rect(left=307, top=285, width=675, height=56), polygon=[Point(x=307, y=287), Point(x=307, y=291), Point(x=309, y=341), Point(x=634, y=341), Point(x=980, y=340), Point(x=982, y=308), Point(x=982, y=306), Point(x=980, y=286), Point(x=634, y=285)], quality=64, orientation='UP')
Данные: b'2860601473409'



### Применение Гауссовского блюра, перевода картинки в черно-белый режим с добавлением бинаризации по порогу и усилением порога

In [32]:
image = cv2.imread(FILE_PATH + bad_barcodes_list_1[3])
image = image[0:image.shape[0], 0:int(image.shape[1]*0.25)]
image = cv2.GaussianBlur(image, (1, 1), 0)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image = cv2.threshold(image, 197, 255, cv2.THRESH_BINARY)

_, img = decode(image)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Обнаружен штрих-код:
Decoded(data=b'2860610411164', type='EAN13', rect=Rect(left=229, top=117, width=658, height=31), polygon=[Point(x=229, y=131), Point(x=229, y=137), Point(x=230, y=147), Point(x=570, y=148), Point(x=887, y=142), Point(x=887, y=118), Point(x=230, y=117)], quality=29, orientation='UP')
Данные: b'2860610411164'

